Edge index excluded from dataloader (used it as a global variable)

In [2]:
import os ; os.chdir(r"D:\Git_repos\Photonic_GCN")
from constants import edges_table, nodes_groups
# !pip install torch torch-geometric
import torch , pandas as pd
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.utils.data import TensorDataset, random_split , DataLoader as DataLoader_primary
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


Inputs

In [8]:
BATCH_SIZE = 8

Edges list declaration

In [3]:
def edges_table_to_list(edges_table):
    edges = []
    for key in edges_table.keys():
        nodes_list = edges_table[key]
        for node in nodes_list:
            if ((key, node) not in edges) and ((node, key) not in edges):
                edges.append((key, node))
    return edges

In [4]:
edge_list = edges_table_to_list(edges_table)  # 107 edges
print(len(edge_list)) 
print(edge_list)

107
[('N69', 'N70'), ('N70', 'N71'), ('N71', 'N72'), ('N72', 'N73'), ('N73', 'N74'), ('N73', 'N60'), ('N74', 'N60'), ('N74', 'N94'), ('N60', 'N45'), ('N60', 'N46'), ('N60', 'N47'), ('N60', 'N61'), ('N60', 'N75'), ('N41', 'N42'), ('N42', 'N43'), ('N43', 'N44'), ('N44', 'N45'), ('N45', 'N46'), ('N46', 'N33'), ('N87', 'N86'), ('N86', 'N85'), ('N85', 'N84'), ('N84', 'N83'), ('N83', 'N82'), ('N83', 'N68'), ('N82', 'N68'), ('N82', 'N94'), ('N68', 'N55'), ('N68', 'N54'), ('N68', 'N53'), ('N68', 'N67'), ('N68', 'N81'), ('N59', 'N58'), ('N58', 'N57'), ('N57', 'N56'), ('N56', 'N55'), ('N55', 'N54'), ('N54', 'N34'), ('N33', 'N61'), ('N33', 'N47'), ('N33', 'N48'), ('N34', 'N52'), ('N34', 'N53'), ('N34', 'N67'), ('N94', 'N61'), ('N94', 'N75'), ('N94', 'N76'), ('N95', 'N80'), ('N95', 'N81'), ('N95', 'N82'), ('N95', 'N67'), ('N67', 'N81'), ('N67', 'N53'), ('N67', 'N80'), ('N67', 'N66'), ('N67', 'N52'), ('N53', 'N52'), ('N53', 'N66'), ('N81', 'N80'), ('N81', 'N66'), ('N80', 'N79'), ('N80', 'N66'), ('N

In [5]:
node_names = list(edges_table.keys())  # 51 nodes
print(node_names)

['N69', 'N70', 'N71', 'N72', 'N73', 'N74', 'N60', 'N41', 'N42', 'N43', 'N44', 'N45', 'N46', 'N87', 'N86', 'N85', 'N84', 'N83', 'N82', 'N68', 'N59', 'N58', 'N57', 'N56', 'N55', 'N54', 'N33', 'N34', 'N94', 'N95', 'N67', 'N53', 'N81', 'N80', 'N52', 'N79', 'N51', 'N65', 'N66', 'N78', 'N64', 'N50', 'N61', 'N75', 'N47', 'N62', 'N76', 'N48', 'N63', 'N77', 'N49']


In [6]:
# Convert edge list to indices
edge_index = torch.tensor([[node_names.index(u), node_names.index(v)] for u, v in edge_list], dtype=torch.long).t().contiguous()
edge_index_orignal = edge_index

edge_index.shape

torch.Size([2, 107])

Data preparation

In [41]:
df = pd.read_excel('data.xlsx')

X = df.iloc[:, :18].values
y = df.iloc[:, 18:19].values

scaler_X, scaler_y = MinMaxScaler(), MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32).to(device)

train_size = int(0.7 * len(X_tensor))
val_size = int(0.15 * len(X_tensor))
test_size = len(X_tensor) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(TensorDataset(X_tensor, y_tensor), [train_size, val_size, test_size])

train_dataloader = DataLoader_primary(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader_primary(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader_primary(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [42]:
print(X.shape)
df

(8015, 18)


,dx1,dx2,dx3,dx4,dx5,dx6,dy1,dy2,dy3,dy4,...,dy6,dr1,dr2,dr3,dr4,dr5,dr6,Q,t_q_max,lambda_q_max
0,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,918.130324,0.554622,0.000001
1,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.750000e-08,2.500000e-08,1846.114296,0.682024,0.000001
2,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,3.000000e-08,2.500000e-08,660.817503,1.089219,0.000002
3,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.750000e-08,2.500000e-08,1846.114296,0.682024,0.000001
4,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.500000e-08,3.500000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.750000e-08,2.500000e-08,1846.114296,0.682024,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8010,-4.000000e-08,3.500000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,1204.550299,0.521705,0.000001
8011,-4.000000e-08,4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,1306.557266,0.549646,0.000001
8012,-4.000000e-08,4.500000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,618.706366,0.701339,0.000002
8013,-4.000000e-08,5.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,-4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,4.000000e-08,...,4.000000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,2.500000e-08,610.208722,0.826126,0.000002


data normalization

dx , dy are between 1.55e-7 and -4e-8 - dr is between 1.75e-8 and 3.25e-8
Almost 98% of Qs are between 500 and 50000

we multiply dx, dy & dr by 1e+7 and Q by 1e-4

In [53]:
normalized_df = df.copy()

normalized_df.iloc[:, 0:18] = normalized_df.iloc[:, 0:18] * 1e+7
normalized_df.iloc[:, 18] = normalized_df.iloc[:, 18] * 1e-4
normalized_df

,dx1,dx2,dx3,dx4,dx5,dx6,dy1,dy2,dy3,dy4,...,dy6,dr1,dr2,dr3,dr4,dr5,dr6,Q,t_q_max,lambda_q_max
0,-0.4,-0.40,-0.4,-0.4,-0.4,-0.45,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.250,0.25,0.091813,0.554622,0.000001
1,-0.4,-0.40,-0.4,-0.4,-0.4,-0.45,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.275,0.25,0.184611,0.682024,0.000001
2,-0.4,-0.40,-0.4,-0.4,-0.4,-0.45,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.300,0.25,0.066082,1.089219,0.000002
3,-0.4,-0.40,-0.4,-0.4,-0.4,-0.45,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.275,0.25,0.184611,0.682024,0.000001
4,-0.4,-0.40,-0.4,-0.4,-0.4,-0.45,0.35,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.275,0.25,0.184611,0.682024,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8010,-0.4,0.35,-0.4,-0.4,-0.4,-0.40,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.250,0.25,0.120455,0.521705,0.000001
8011,-0.4,0.40,-0.4,-0.4,-0.4,-0.40,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.250,0.25,0.130656,0.549646,0.000001
8012,-0.4,0.45,-0.4,-0.4,-0.4,-0.40,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.250,0.25,0.061871,0.701339,0.000002
8013,-0.4,0.50,-0.4,-0.4,-0.4,-0.40,0.40,0.4,0.4,0.4,...,0.4,0.25,0.25,0.25,0.25,0.250,0.25,0.061021,0.826126,0.000002


In [54]:
nodes2group = dict()

for g in nodes_groups:
    for node in nodes_groups[g]:
        nodes2group[node] = g

In [55]:
xs_list = []
ys_list = []

for n in range(len(df)):
    x = np.zeros((51, 3))
    y = np.array(normalized_df["Q"][n])
    for idx, node in enumerate(node_names):
        group = nodes2group[node][1:]
        node_features = np.array((normalized_df["dx" + group][n], normalized_df["dy" + group][n], normalized_df["dr" + group][n],))
        x[idx] = node_features
    xs_list.append(x)
    ys_list.append(y)

data_x = torch.from_numpy(np.stack(xs_list, axis=0))
data_y = torch.from_numpy(np.stack(ys_list, axis=0))
print(data_x.shape)
print(data_y.shape)
print(edge_index.shape)

torch.Size([8015, 51, 3])
torch.Size([8015])
torch.Size([2, 107])


In [56]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, x, y):
        """
        Args:
            x (Tensor): Tensor of shape (8015, 51, 3) containing the input data.
            labels (Tensor): Tensor of shape (8015,) containing the labels.
        """
        self.x = x.float()  # Ensure x is float32
        self.y = y.float()  # Ensure y is float32
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [57]:
dataset = MyDataset(data_x, data_y)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [58]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1)  # Output 1 feature per node

    def forward(self, data, edge_index):
        x = data
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(Net, self).__init__()
        self.gcn = GCN(in_channels, hidden_channels)
        self.fc1 = torch.nn.Linear(51, 128)  # Input size should match number of nodes
        self.fc2 = torch.nn.Linear(128, 1)   # Output a single value

    def forward(self, data, edge_index):
        x = self.gcn(data, edge_index)
        x = x.view(1, -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Model parameters
in_channels = 3
hidden_channels = 16

# Initialize model
model = Net(in_channels, hidden_channels)
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [59]:
# edge_index = edge_index.to(device) 

def train(loader, model, optimizer, criterion, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in loader:
            optimizer.zero_grad()
            x, y = batch
            x = x.float()  # Move x to the correct device
            y = y.float()  # Move y to the correct device
            # x.to(device); y.to(device) 
            out = model(x, edge_index)  # Pass edge_index to the model
            y = y.view(out.shape)  # Ensure y matches output shape
            loss = criterion(out, y)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize the model
            total_loss += loss.item()  # Accumulate the loss

        avg_loss = total_loss / len(loader)  # Compute average loss for the epoch
        print(f'Epoch {epoch}, Loss: {avg_loss:.4f}')


train(dataloader, model, optimizer, criterion, epochs=100)

Epoch 0, Loss: 0.0205
Epoch 1, Loss: 0.0200


KeyboardInterrupt: 